In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, LineString
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
#dos librerias nuevas que usamos en APIs
import urllib
import json



# GCBA api

In [3]:
#una forma elemental de consultar una api es usando el navegador
'https://apitransporte.buenosaires.gob.ar/ecobici/gbfs/stationInformation?client_id=02f9fb1291ef41f6bfa93b0d99779291&client_secret=F7e10670D41343E1bf0D3DFFc7aa1e7D'

'https://apitransporte.buenosaires.gob.ar/ecobici/gbfs/stationInformation?client_id=02f9fb1291ef41f6bfa93b0d99779291&client_secret=F7e10670D41343E1bf0D3DFFc7aa1e7D'

In [4]:
url = 'https://apitransporte.buenosaires.gob.ar/ecobici/gbfs/stationInformation?client_id=02f9fb1291ef41f6bfa93b0d99779291&client_secret=F7e10670D41343E1bf0D3DFFc7aa1e7D'

In [5]:
#esto se descompone en varios elementos
url_base = 'https://apitransporte.buenosaires.gob.ar/ecobici/gbfs/stationInformation?'
client_id = 'client_id=02f9fb1291ef41f6bfa93b0d99779291'
client_secret = 'client_secret=F7e10670D41343E1bf0D3DFFc7aa1e7D'

In [6]:
#cada API puede ser diferente y debemos chequear su documentación, pero los diferentes parametros
#se separan en general con &
url_computesta = url_base + client_id + '&' + client_secret
url_computesta

'https://apitransporte.buenosaires.gob.ar/ecobici/gbfs/stationInformation?client_id=02f9fb1291ef41f6bfa93b0d99779291&client_secret=F7e10670D41343E1bf0D3DFFc7aa1e7D'

In [7]:
url_computesta == url

True

En este caso los unicos dos parametros son `client_id` y `client_secret`. Normalmente las APIs piden algun tipo de _token_ o clave secreta **que debemos evitar compartir (como estoy haciendo yo aca)**. En los casos siguientes vamos a ver otros ejemplos con consultas mas complejas

In [12]:
#realizamos el request a esa url
response = urllib.request.urlopen(url)
response

In [13]:
#la libreria json nos permite leer esta informacion como un diccionario de Python
datos = json.loads(response.read())


In [16]:
#inspeccionemos nuestros datos
type(datos)

dict

In [29]:
datos['data']['stations'][0]

{'station_id': '2',
 'name': '002 - Retiro I',
 'physical_configuration': 'SMARTLITMAPFRAME',
 'lat': -34.5924233,
 'lon': -58.3747151,
 'altitude': 0.0,
 'address': 'Ramos Mejia, Jose Maria, Dr. Av. & Del Libertador Av.',
 'post_code': '11111',
 'capacity': 20,
 'rental_methods': ['KEY', 'TRANSITCARD', 'PHONE'],
 'groups': ['RETIRO'],
 'obcn': '',
 'nearby_distance': 1000.0}

In [30]:
pd.DataFrame({'nombres':['felipe','diego'],
             'edades':[15,22]})

,nombres,edades
0,felipe,15
1,diego,22


In [32]:
pd.DataFrame(datos['data']['stations'])

,station_id,name,physical_configuration,lat,lon,altitude,address,post_code,capacity,rental_methods,groups,obcn,nearby_distance,cross_street
0,2,002 - Retiro I,SMARTLITMAPFRAME,-34.592423,-58.374715,0.0,"Ramos Mejia, Jose Maria, Dr. Av. & Del Liberta...",11111,20,"[KEY, TRANSITCARD, PHONE]",[RETIRO],,1000.0,NaN
1,3,003 - ADUANA,SMARTLITMAPFRAME,-34.611032,-58.368260,0.0,Moreno & Av Paseo Colon,11111,20,"[KEY, TRANSITCARD, PHONE]",[MONSERRAT],NaN,1000.0,Moreno & Av Paseo Colon
2,4,004 - Plaza Roma,SMARTLITMAPFRAME,-34.601726,-58.368763,0.0,Lavalle & Bouchard,11111,20,"[KEY, TRANSITCARD, PHONE]",[SAN NICOLAS],,1000.0,NaN
3,5,005 - Plaza Italia,SMARTLITMAPFRAME,-34.580550,-58.420954,0.0,Av. Sarmiento 2601,1111,42,"[KEY, TRANSITCARD, PHONE]",[PALERMO],,1000.0,NaN
4,6,006 - Parque Lezama,SMARTLITMAPFRAME,-34.628526,-58.369758,0.0,"Avenida Martin Garcia, 295",1111,20,"[KEY, TRANSITCARD, PHONE]",[SAN TELMO],,1000.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,444,061-Ministerio de Economia,SMARTLITMAPFRAME,-34.608678,-58.371008,0.0,"Balcarce & Yrigoyen, Hipolito Av.",1111,24,"[KEY, TRANSITCARD, PHONE]",[],,1000.0,NaN
395,448,393 - Barrio 31,SMARTLITMAPFRAME,-34.582618,-58.378731,0.0,Carlos H. Perette 11,1111,16,"[KEY, TRANSITCARD, PHONE]",[RETIRO],,1000.0,NaN
396,449,352 - San Jose de Flores,SMARTLITMAPFRAME,-34.628937,-58.463661,0.0,Avenida Rivadavia y Fray Cayetano,1111,24,"[KEY, TRANSITCARD, PHONE]",[FLORES],,1000.0,NaN
397,450,teste ecobici - 450,VALET,-23.556362,-46.662562,NaN,teste ecobici - 450,NaN,1,"[KEY, TRANSITCARD]",[],000204,1000.0,NaN


In [33]:
#dado el formato en el que se 
estaciones = pd.DataFrame(datos['data']['stations'])
estaciones = estaciones.reindex(columns=['station_id', 'name', 'capacity', 'lat', 'lon'])
estaciones.head()

,station_id,name,capacity,lat,lon
0,2,002 - Retiro I,20,-34.592423,-58.374715
1,3,003 - ADUANA,20,-34.611032,-58.368260
2,4,004 - Plaza Roma,20,-34.601726,-58.368763
3,5,005 - Plaza Italia,42,-34.580550,-58.420954
4,6,006 - Parque Lezama,20,-34.628526,-58.369758


In [34]:
url_base = 'https://apitransporte.buenosaires.gob.ar/ecobici/gbfs/stationStatus?'
url_computesta = url_base + client_id + '&' + client_secret
url_computesta

'https://apitransporte.buenosaires.gob.ar/ecobici/gbfs/stationStatus?client_id=02f9fb1291ef41f6bfa93b0d99779291&client_secret=F7e10670D41343E1bf0D3DFFc7aa1e7D'

In [39]:
#conusltamos el status de las estaciones
response = urllib.request.urlopen(url_computesta)
status = json.loads(response.read())
status = pd.DataFrame(status['data']['stations'])


In [40]:
status

,station_id,num_bikes_available,num_bikes_available_types,num_bikes_disabled,num_docks_available,num_docks_disabled,is_installed,is_renting,is_returning,last_reported,is_charging_station,status
0,2,0,"{'mechanical': 0, 'ebike': 0}",5,15,0,1,1,1,1574892733,False,IN_SERVICE
1,3,0,"{'mechanical': 0, 'ebike': 0}",1,19,0,1,1,1,1574892740,False,IN_SERVICE
2,4,0,"{'mechanical': 0, 'ebike': 0}",0,20,1,1,1,1,1574892733,False,IN_SERVICE
3,5,0,"{'mechanical': 0, 'ebike': 0}",1,41,0,1,1,1,1574892698,False,IN_SERVICE
4,6,0,"{'mechanical': 0, 'ebike': 0}",2,18,0,1,1,1,1574892740,False,IN_SERVICE
...,...,...,...,...,...,...,...,...,...,...,...,...
411,444,1,"{'mechanical': 1, 'ebike': 0}",0,23,1,1,1,1,1574892622,False,IN_SERVICE
412,448,4,"{'mechanical': 4, 'ebike': 0}",1,11,2,1,1,1,1574892697,False,IN_SERVICE
413,449,0,"{'mechanical': 0, 'ebike': 0}",3,21,0,1,1,1,1574892624,False,IN_SERVICE
414,450,198,"{'mechanical': 99, 'ebike': 99}",0,99,0,1,1,1,1567199965,False,IN_SERVICE


In [43]:
datetime.now()

datetime.datetime(2019, 11, 27, 19, 17, 35, 557416)

In [42]:
#extraemos el momento en que consultamos los datos, especificando que nuestro timezone es el de Argentina
from datetime import datetime
import pytz

argentina_tz = pytz.timezone('America/Argentina/Buenos_Aires')
tiempo_consulta = datetime.now(tz=argentina_tz)

#conusltamos el status de las estaciones
response = urllib.request.urlopen(url_computesta)
status = json.loads(response.read())
status = pd.DataFrame(status['data']['stations'])

#mergeamos ambos datasets para tener para cada status, la informacion de la estacion
status = status.merge(estaciones, how='inner', on='station_id')
status['timestamp_consulta'] = tiempo_consulta
status.to_csv('../data/' + str(tiempo_consulta)[:16] + '.csv', index=False)


In [44]:
status.head()

,station_id,num_bikes_available,num_bikes_available_types,num_bikes_disabled,num_docks_available,num_docks_disabled,is_installed,is_renting,is_returning,last_reported,is_charging_station,status,name,capacity,lat,lon,timestamp_consulta
0,2,0,"{'mechanical': 0, 'ebike': 0}",5,15,0,1,1,1,1574893002,False,IN_SERVICE,002 - Retiro I,20,-34.592423,-58.374715,2019-11-27 19:17:29.902426-03:00
1,3,0,"{'mechanical': 0, 'ebike': 0}",0,20,0,1,1,1,1574892996,False,IN_SERVICE,003 - ADUANA,20,-34.611032,-58.368260,2019-11-27 19:17:29.902426-03:00
2,4,0,"{'mechanical': 0, 'ebike': 0}",0,20,1,1,1,1,1574893000,False,IN_SERVICE,004 - Plaza Roma,20,-34.601726,-58.368763,2019-11-27 19:17:29.902426-03:00
3,5,0,"{'mechanical': 0, 'ebike': 0}",1,41,0,1,1,1,1574892966,False,IN_SERVICE,005 - Plaza Italia,42,-34.580550,-58.420954,2019-11-27 19:17:29.902426-03:00
4,6,0,"{'mechanical': 0, 'ebike': 0}",2,18,0,1,1,1,1574893005,False,IN_SERVICE,006 - Parque Lezama,20,-34.628526,-58.369758,2019-11-27 19:17:29.902426-03:00


Aca pueden ver una funcion que hace esto y se se deja corriendo consulta y guarda cada media hora

In [ ]:
#https://github.com/alephcero/ecobici_un_dia
import os
import urllib.request as urllib
import json
import pandas as pd
from datetime import datetime
from time import sleep
import pytz


argentina_tz = pytz.timezone('America/Argentina/Buenos_Aires')


def consultar_y_persistir():
    url_datos = 'https://apitransporte.buenosaires.gob.ar/ecobici/gbfs/stationInformation?client_id=' + \
        os.environ['ECOBICIID'] + '&client_secret=' + \
        os.environ['ECOBICISECRET']
    response = urllib.urlopen(url_datos)
    datos = json.loads(response.read())
    estaciones = pd.DataFrame(datos['data']['stations'])
    estaciones = estaciones.reindex(
        columns=['station_id', 'name', 'capacity', 'lat', 'lon'])

    url_consulta = 'https://apitransporte.buenosaires.gob.ar/ecobici/gbfs/stationStatus?client_id=' + \
        os.environ['ECOBICIID'] + '&client_secret=' + \
        os.environ['ECOBICISECRET']
    argentina_tz = pytz.timezone('America/Argentina/Buenos_Aires')
    tiempo_consulta = datetime.now(tz=argentina_tz)
    response = urllib.urlopen(url_consulta)
    status = json.loads(response.read())
    dt = pd.DataFrame(status['data']['stations'])
    dt = dt.merge(estaciones, how='inner', on='station_id')
    dt['timestamp_consulta'] = tiempo_consulta
    dt.to_csv('data/' + str(tiempo_consulta)[:16] + '.csv', index=False)


no_es_medianoche = True
while no_es_medianoche:
    if (datetime.now(tz=argentina_tz).hour == 0) and (datetime.now(tz=argentina_tz).minute == 0):
        no_es_medianoche = False
    print("waiting", datetime.now(tz=argentina_tz))
    sleep(60 * 1)

for i in range(48):
    sleep(60 * 30)
    print(i)
    consultar_y_persistir()

# Google

In [ ]:
#Primero creamos un dataset para testear
#formato google maps latlong, yx
obelisco = (-34.60377,-58.38161)
retiro = (-34.591372, -58.374261)
once = (-34.608315, -58.406169)
constitucion = (-34.627554, -58.380690)

def construir_dict(nombre,tupla_latlong):
    return {
        'nombre':nombre,
        'coordenadas':{
            'long':tupla_latlong[1],
            'lat':tupla_latlong[0],
            'x':tupla_latlong[1],
            'y':tupla_latlong[0]
        }
}
    
puntos = [construir_dict('obelisco',(-34.60377,-58.38161)),
         construir_dict('retiro',(-34.591372, -58.374261)),
         construir_dict('once',(-34.608315, -58.406169)),
         construir_dict('constitucion',(-34.627554, -58.380690))]
crs = "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"
geoserie = gpd.GeoSeries([Point(punto['coordenadas']['x'],punto['coordenadas']['y']) for punto in puntos],crs=crs) 
geodt = gpd.GeoDataFrame({'geometry': geoserie, 'nombre':[punto['nombre'] for punto in puntos]})
geodt.index=geodt.nombre
geodt

In [ ]:
# veamos la documentacion para obtener un token 
# https://developers.google.com/maps/documentation/distance-matrix/get-api-key
# a esta documentacion le falta el paso de habilitar ENABLE
api = ''

Nuestra itencion es obtener tiempos de viaje por eso usaremos Distance Matrix API.


In [ ]:
# leamos la documentacion

In [ ]:
#https://developers.google.com/maps/documentation/distance-matrix/intro#DistanceMatrixRequests

In [ ]:
#creamos nuestras coordenadas en el formato requerido por la API de google
origen_lat = geodt.loc['obelisco','geometry'].y
origen_long = geodt.loc['obelisco','geometry'].x
destino_lat = geodt.loc['retiro','geometry'].y
destino_long = geodt.loc['retiro','geometry'].x

In [ ]:
#construimos la consulta con el mismo metodo que vimos antes
url_base = 'https://maps.googleapis.com/maps/api/distancematrix/json?'
units = 'units=metric' 
origins = 'origins=' + str(origen_lat) + ',' + str(origen_long)
destinations = 'destinations=%f,%f' % (destino_lat,destino_long)
key = 'key={0}'.format(api)

url = url_base + '&' + units + '&' + origins + '&' + destinations + '&' + key
url

In [ ]:
#volvemos a realizar la consulta y leerla como diccionario de python
response = urllib.request.urlopen(url)
diccio = json.loads(response.read())
diccio

In [ ]:
#podemos agregar un nuevo parametro, el modo de transporte y agregarlo a lo que ya veniamos consultando
mode = 'mode=bicycling'

url = url_base + '&' + units + '&' + origins + '&' + destinations + '&'+ mode + '&' + key
url

In [ ]:
#repetimos el proceso de consulta
response = urllib.request.urlopen(url)
diccio_bici = json.loads(response.read())
diccio_bici

In [ ]:
#afortunadamente la gente de google creo una libreria que hace todo esto por nosotros
#https://github.com/googlemaps/google-maps-services-python
import googlemaps

In [ ]:
#documentacion
#https://github.com/googlemaps/google-maps-services-python
gmaps = googlemaps.Client(key=api)

In [ ]:
#realizamos una consulta con la liberia
travel = gmaps.distance_matrix(
    origins = once,
    destinations = obelisco,
    mode='transit', traffic_model='best_guess',
    transit_routing_preference = 'fewer_transfers')

In [ ]:
#vemos el resultado de nuestro consulta
travel

In [ ]:
#vemos que es un diccionario
type(travel)

In [ ]:
#podemos convertirlo en un pandas data frame a partir de la estructura de diccionario
data = pd.DataFrame(travel)
data

In [ ]:
#como extraer la informacion que deseamos?